<a href="https://colab.research.google.com/github/dduniverse/ML-study/blob/main/09_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RandomForest, 랜덤 포레스트
랜덤 포레스트(Random Forest): 결정 트리의 단점인 오버피팅 문제를 완화시켜주는 발전된 형태의 트리 모델
- 랜덤으로 독립적인 트리를 여러 개 만들어서 예측
- 여러 모델을 활용하여 하나의 모델을 이루는 기법을 앙상블이라 함

|장점|단점|
|---|---|
|결정 트리와 마찬가지로, 아웃라이어에 거의 영향을 받지 않음|학습 속도가 상대적으로 느림|
|선형/비선형 데이터에 상관없이 잘 작동함|수많은 트리를 동원하기 떄문에 모델에 대한 해석이 어려움|


- 종속변수가 연속형 데이터와 범주형 데이터인 경우 모두 사용가능
- 아웃라이어가 문제가 되는 경우 선형 모델보다 좋은 대안이 될 수 있음
- 오버피팅 문제로 결정트리를 사용하기 어려울 때 사용 가능

## 라이브러리 및 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as plt

In [2]:
file_url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
data = pd.read_csv(file_url)
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [3]:
data.info()  # 컬럼 정보 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [4]:
round(data.describe(), 2)

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


- 결측치 존재
- enigne, max_power 등 숫자형이 더 적합하나 문자형 데이터로 되어 있음을 알 수 있음
- 아웃라이어가 존재함 -> 트리 모델을 사용하므로 별도의 처리를 하지 않음
  - selling_price에서 max가 유독 높음
  - km_driven은 max와 min 모두 아웃라이어로 보임


## 전처리: 텍스트 데이터

### 1. engine 변수 전처리
- split() 함수로 숫자와 문자를 각각 더미 변수로 생성
- astype()으로 0열을 숫자형으로 변환
- 1열 값은 모두 'CC' 이므로 drop으로 제거

In [5]:
data['engine'].str.split(expand=True)  # 공백 기준으로 문자 분리

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [6]:
data[['engine', 'engine_unit']] = data['engine'].str.split(expand=True)

In [7]:
data['engine'].head()

0    1248
1    1498
2    1497
3    1396
4    1298
Name: engine, dtype: object

In [8]:
data['engine'] = data['engine'].astype('float32')  # 숫자형으로 변환

In [9]:
data['engine'].head()

0    1248.0
1    1498.0
2    1497.0
3    1396.0
4    1298.0
Name: engine, dtype: float32

In [10]:
data['engine_unit'].unique()  # 고윳값 확인 - 단위 확인 과정

array(['CC', nan], dtype=object)

In [11]:
data.drop('engine_unit', axis=1, inplace=True)  # 변수 제거

### 2. max_power 변수 전처리

In [12]:
data[['max_power', 'max_power_unit']] = data['max_power'].str.split(expand=True)

In [13]:
data['max_power'].head()

0        74
1    103.52
2        78
3        90
4      88.2
Name: max_power, dtype: object

In [14]:
data['max_power'] = data['max_power'].astype('float32')  # 숫자형으로 변환

ValueError: ignored

In [15]:
data[data['max_power'] == 'bhp'] 

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0,None


원래 데이터에 숫자 없이 'bhp'만 있어서 str.split()에서 분리되지 않은 채로 남음
- try and except 구문을 활용하여 처리

In [16]:
def isFloat(value):
  try:
    num = float(value)
    return num
  except ValueError:  # ValueError가 발생하면 np.NaN 리턴
    return np.NaN

In [17]:
data['max_power'] = data['max_power'].apply(isFloat)  # isFloat 함수를 사용하여 숫자형으로 변환

In [18]:
data['max_power_unit'].unique()  # 고윳값 확인 - 단위 확인

array(['bhp', nan, None], dtype=object)

In [19]:
data.drop('max_power_unit', axis=1, inplace=True)  # 변수 제거

### 3. mileage 변수 전처리

In [20]:
data[['mileage', 'mileage_unit']] = data['mileage'].str.split(expand=True)

In [21]:
data['mileage'] = data['mileage'].astype('float32')  # 숫자형으로 변환

In [22]:
data['mileage_unit'].unique()  # 고윳값 확인 - 단위가 2개 존재

array(['kmpl', 'km/kg', nan], dtype=object)

- kmpl: 리터당 킬로미터(=km/l) - 휘발유/디젤 측정 단위
- km/kg: 킬로그램당 킬로미터 - 연료 측정 단위

근본적으로 연료의 종류 때문에 발생한 문제이므로 fuel 컬럼을 활용하여 해결

In [23]:
data['fuel'].unique()  # 고윳값 확인 - 연료 종류

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

동일 시점의 가격
- Petrol: 리터당 $80.43
- Diesel: 리터당 $73.56
- LPG: 킬로그램당 $40.85
- CNG: 킬로그램당 $44.23

mileage를 각 연료별 가격으로 나누면 1달러당 주행거리가 됨

In [24]:
def mile(x):
  if x['fuel'] == 'Petrol':
    return x['mileage'] / 80.43
  elif x['fuel'] == 'Diesel':
    return x['mileage'] / 73.56
  elif x['fuel'] == 'LPG':
    return x['mileage'] / 40.85
  else:
    return x['mileage'] / 44.23

In [25]:
data['mileage'] = data.apply(mile, axis=1)  # mile 함수를 이용하여 mileage 수정

In [26]:
data.drop('mileage_unit', axis=1, inplace=True)  # 변수 제거

### 4. torque 변수 전처리
- 단위 앞부분 숫자만 추출해 숫자형으로 바꿈
- 단위가 Nm인지 kgm인지 확인
   - kgm이면 9.8066을 곱해 단위를 Nm으로 통일
   - 결측치는 Nm으로 대체

In [27]:
data['torque'].head()

0              190Nm@ 2000rpm
1         250Nm@ 1500-2500rpm
2       12.7@ 2,700(kgm@ rpm)
3    22.4 kgm at 1750-2750rpm
4       11.5@ 4,500(kgm@ rpm)
Name: torque, dtype: object

In [28]:
data['torque'] = data['torque'].str.upper()  # torque 변수 대문자로 변환
data['torque']

0                  190NM@ 2000RPM
1             250NM@ 1500-2500RPM
2           12.7@ 2,700(KGM@ RPM)
3        22.4 KGM AT 1750-2750RPM
4           11.5@ 4,500(KGM@ RPM)
                  ...            
8123             113.7NM@ 4000RPM
8124    24@ 1,900-2,750(KGM@ RPM)
8125               190NM@ 2000RPM
8126          140NM@ 1800-3000RPM
8127          140NM@ 1800-3000RPM
Name: torque, Length: 8128, dtype: object

In [29]:
def torque_unit(x):  # 단위 반환 함수
  if 'NM' in str(x):
    return 'Nm'
  elif 'KGM' in str(x):
    return 'kgm'

In [30]:
data['torque_unit'] = data['torque'].apply(torque_unit)  # torque_unit 함수를 사용하여 단위 반환
data['torque_unit'].unique()  # 고윳값 확인 - 단위 종류

array(['Nm', 'kgm', None], dtype=object)

In [31]:
data[data['torque_unit'].isna()]  # torque_unit이 null값인 데이터 확인

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,torque_unit
13,Maruti Swift 1.3 VXi,2007,200000,80000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
31,Fiat Palio 1.2 ELX,2003,70000,50000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
78,Tata Indica DLS,2003,50000,70000,Diesel,Individual,Manual,First Owner,NaN,NaN,NaN,NaN,NaN,None
87,Maruti Swift VDI BSIV W ABS,2015,475000,78000,Diesel,Dealer,Manual,First Owner,NaN,NaN,NaN,NaN,NaN,None
119,Maruti Swift VDI BSIV,2010,300000,120000,Diesel,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7846,Toyota Qualis Fleet A3,2000,200000,100000,Diesel,Individual,Manual,First Owner,NaN,NaN,NaN,NaN,NaN,None
7996,Hyundai Santro LS zipPlus,2000,140000,50000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
8009,Hyundai Santro Xing XS eRLX Euro III,2006,145000,80000,Petrol,Individual,Manual,Second Owner,NaN,NaN,NaN,NaN,NaN,None
8068,Ford Figo Aspire Facelift,2017,580000,165000,Diesel,Individual,Manual,First Owner,NaN,NaN,NaN,NaN,NaN,None


In [32]:
data[data['torque_unit'].isna()]['torque'].unique()  # torque_unit이 결측치인 데이터의 torque 고윳값 확인

array([nan, '250@ 1250-5000RPM', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

위 데이터가 Nm과 kgm 중 하나에 속한다는 전제하에
- Nm은 100, 200 등 백 단위 숫자
- kgm은 10, 20 등 십 단위 숫자

즉, 숫자 크기를 고려하면 모두 Nm에 해당함을 추론할 수 있음

In [33]:
data['torque_unit'].fillna('Nm', inplace=True)  # 결측치를 Nm으로 대체

In [34]:
string_example = '12.7@ 2,700(KGM@ RPM)'
for i, j in enumerate(string_example):
  if j not in '0123456789.':  # 숫자도 마침표도 아닌 것 중 가장 첫 번째로 나타나는 j의 자리(=i)
    print(i)
    break

4


In [35]:
def split_num(x):
  x = str(x)  # 문자형태로 변환
  for i, j in enumerate(x):
    if j not in '0123456789.':
      cut = i
      break
  return x[:cut]  # cut 이전 자리까지 인덱싱하여 리턴

In [36]:
data['torque'] = data['torque'].apply(split_num)  # split_num 함수를 적용하여 숫자값만 추출하기

In [37]:
data['torque']

0         190
1         250
2        12.7
3        22.4
4        11.5
        ...  
8123    113.7
8124       24
8125      190
8126      140
8127      140
Name: torque, Length: 8128, dtype: object

In [38]:
data['torque'] = data['torque'].astype('float64')  # 숫자형으로 변환

ValueError: ignored

비어있는 string 값 ''이 존재하기 때문에 숫자형으로 한꺼번에 변환이 되지 않음
- 앞서 진행한 방법처럼 try and except 구문을 사용할 수 있음
- 간단히 null로 대체

In [39]:
data['torque'] = data['torque'].replace('', np.NaN)  # ''을 NaN으로 대체

In [40]:
data['torque'] = data['torque'].astype('float64')

In [41]:
data['torque'].head()

0    190.0
1    250.0
2     12.7
3     22.4
4     11.5
Name: torque, dtype: float64

In [42]:
def torque_trans(x):
  if x['torque_unit'] == 'kgm':  # 단위가 kgm이면 9.8066을 곱해 리턴
    return x['torque'] * 9.8066
  else:
    return x['torque']

In [43]:
data['torque'] = data.apply(torque_trans, axis=1)
data['torque']

0       190.00000
1       250.00000
2       124.54382
3       219.66784
4       112.77590
          ...    
8123    113.70000
8124    235.35840
8125    190.00000
8126    140.00000
8127    140.00000
Name: torque, Length: 8128, dtype: float64

In [44]:
data.drop('torque_unit', axis=1, inplace=True)  # 변수 제거

In [45]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,0.318108,1248.0,74.00,190.00000,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,0.287384,1498.0,103.52,250.00000,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,0.220067,1497.0,78.00,124.54382,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,0.312670,1396.0,90.00,219.66784,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,0.200174,1298.0,88.20,112.77590,5.0


### 5. name 변수 전처리
name: 자동차의 브랜드/모델명이 기입된 변수
- 전체를 더미 변수를 만들면 너무 많은 컬럼이 추가되어 좋지 않음
- 동일 스펙의 자동차라도 비싼 브랜드의 경우 가격이 더 비쌀 수 있으므로 모델명은 버리더라도 브랜드명은 가져가야함

In [46]:
data['name']

0             Maruti Swift Dzire VDI
1       Skoda Rapid 1.5 TDI Ambition
2           Honda City 2017-2020 EXi
3          Hyundai i20 Sportz Diesel
4             Maruti Swift VXI BSIII
                    ...             
8123               Hyundai i20 Magna
8124           Hyundai Verna CRDi SX
8125          Maruti Swift Dzire ZDi
8126                 Tata Indigo CR4
8127                 Tata Indigo CR4
Name: name, Length: 8128, dtype: object

In [47]:
data['name'] = data['name'].str.split(expand=True)[0]

In [48]:
data['name'].unique()  # 고윳값 확인

array(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW',
       'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo',
       'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot'],
      dtype=object)

Land Rover가 띄어쓰기로 인해 Land만 저장됨
- 모델링에는 영향이 없으나 이름을 변경해줌

In [49]:
data['name'] = data['name'].replace('Land', 'Land Rover')  # Land를 Land Rover로 대체

## 전처리: 결측치 처리와 더미 변수 변환

In [50]:
data.isnull().mean()  # 결측치 비율 확인

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          0.027190
engine           0.027190
max_power        0.026575
torque           0.027313
seats            0.027190
dtype: float64

결측치의 비율이 2% 수준으로 높지 않기 때문에 결측치가 있는 행을 모두 제거해줌

In [51]:
data.dropna(inplace=True)
len(data)

7906

In [52]:
data = pd.get_dummies(data, columns = ['name', 'fuel', 'seller_type', 'transmission', 'owner'], drop_first=True)  # 더미 변수로 변환

In [53]:
data.head()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0


## 모델링 및 평가

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('selling_price', axis=1), data['selling_price'], test_size=0.2, random_state=100)  # train/test 분리

In [55]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=100)
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [56]:
from sklearn.metrics import mean_squared_error
print('train_rmse:', mean_squared_error(y_train, train_pred) ** 0.5, 'test_rmse:', mean_squared_error(y_test, test_pred) ** 0.5)  # rmse 확인

train_rmse: 53531.41548125947 test_rmse: 131855.18391308116


## K-Fold 교차검증
교차검증: 다양한 train/test set을 통하여 모델에 더 신뢰할 수 있는 평가를 하는 방법

K-Fold 교차검증: 데이터를 특정 개수(K개)로 쪼개어 그중 하나씩을 선택하여 시험셋으로 사용하되, 이 과정을 K번만큼 반복
- K번의 모델링 및 평가를 통해 얻어진 K개의 오차를 평균내어 최종 평균값(RMSE)를 도출

In [57]:
data

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,2013,320000,110000,0.230014,1197.0,82.85,113.70000,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
8124,2007,135000,119000,0.228385,1493.0,110.00,235.35840,5.0,0,0,...,1,0,0,1,0,1,1,0,0,0
8125,2009,382000,120000,0.262371,1248.0,73.90,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
8126,2013,290000,25000,0.320419,1396.0,70.00,140.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0


데이터의 인덱스를 보면 0부터 8127까지 있지만 실제로는 7906줄이므로 `reset_index()`를 사용하여 인덱스 정리
- KFold는 중간에 빈 값이 존재하면 에러가 발생함

In [58]:
data.reset_index(drop=True, inplace=True)
data

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7901,2013,320000,110000,0.230014,1197.0,82.85,113.70000,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
7902,2007,135000,119000,0.228385,1493.0,110.00,235.35840,5.0,0,0,...,1,0,0,1,0,1,1,0,0,0
7903,2009,382000,120000,0.262371,1248.0,73.90,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
7904,2013,290000,25000,0.320419,1396.0,70.00,140.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0


In [59]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
X = data.drop('selling_price', axis=1)
y = data['selling_price']

In [60]:
for i, j in kf.split(X):
  print(i, j)

[1582 1583 1584 ... 7903 7904 7905] [   0    1    2 ... 1579 1580 1581]
[   0    1    2 ... 7903 7904 7905] [1582 1583 1584 ... 3160 3161 3162]
[   0    1    2 ... 7903 7904 7905] [3163 3164 3165 ... 4741 4742 4743]
[   0    1    2 ... 7903 7904 7905] [4744 4745 4746 ... 6322 6323 6324]
[   0    1    2 ... 6322 6323 6324] [6325 6326 6327 ... 7903 7904 7905]


In [61]:
train_rmse_total = []
test_rmse_total = []

for train_index, test_index in kf.split(X):
  # X_train, X_test, y_train, y_test 정의
  X_train, X_test = X.loc[train_index], X.loc[test_index]  # X는 데이터프레임 형태이므로 loc를 사용해 인덱싱
  y_train, y_test = y[train_index], y[test_index]  # y는 시리즈 형태이므로 loc를 사용하지 않고 곧바로 인덱싱

  model = RandomForestRegressor(random_state=100)
  model.fit(X_train, y_train)
  train_pred = model.predict(X_train)
  test_pred = model.predict(X_test)

  train_rmse = mean_squared_error(y_train, train_pred) ** 0.5  # train set rmse
  test_rmse = mean_squared_error(y_test, test_pred) ** 0.5  # test set rmse

  train_rmse_total.append(train_rmse)
  test_rmse_total.append(test_rmse)

In [62]:
train_rmse_total

[50825.5556350298,
 58854.04054344074,
 57904.19615940739,
 56218.23740006373,
 58967.150857632456]

In [63]:
print('train_rmse:', sum(train_rmse_total)/5, 'test_rmse:', sum(test_rmse_total)/5)

train_rmse: 56553.836119114814 test_rmse: 142936.58918244042


교차검증을 사용하기 전보다 전반적으로 RMSE가 올라간 것을 볼 수 있음
- 교차검증을 사용한 결과가 조금 더 정확한 평가 결과임

## 모델 이해하기
랜덤 포레스트는 결정 트리의 집합체로 다양한 트리의 의견을 반영하기 때문에 오버피팅 위험을 낮출 수 있음
- 랜덤 포레스트가 트리를 만들 때는 데이터 전체를 사용하지 않고 매번 다른 일부의 데이터를 사용하여 다른 트리를 만들어냄
- 당연히 일부 데이터만 사용하면 전체 데이터를 사용한 결과보다 예측력이 떨어지지만, 예측력이 떨어지는 수많은 트리들이 함게 모여 중윗값을 찾아내면 오버피팅을 막는데 효율적임


랜덤 포레스트가 일부 데이터를 취하는 기준
1. 데이터의 행 기준 - 매번 약 2/3에 해당하는 데이터만 사용
2. 데이터의 열 기준 - 일부의 변수들만 매번 다르게 추출하여 사용


최종 예측값
- 각 트리의 예측값을 기반으로 만들어짐
- 회귀 문제: 연속형 변수를 예측하기 떄문에, 각 트리에서 만들어낸 예측값들의 평균값을 최종 예측값으로 사용
- 분류 문제: 각 트리에서 예측한 값들 중 최다 투푯값으로 예측값 결정

## 하이퍼파라미터 튜닝
- n_estimator: 랜덤 포레스트를 구성하는 결정 트리의 개수(기본값=100)
- max_depth: 각 트리의 최대 깊이(숫자가 낮을 수록 오버피팅을 피할 수 있으나, 언더피팅의 위험이 올라감)
- min_samples_split: 해당 노드를 나눌 것인지 말 것인지를 노드 데이터 수를 기준으로 판단(이 매개변수에 지정된 숫자보다 적은 수의 데이터가 노드에 있으면 더는 분류하지 않음, 기본값=2)
- n_jobs: 병렬 처리에 사용되는 CPU 코어 수(많은 코어를 사용할수록 속도는 상승하고 -1을 입력하면 모든 코어를 사용, 기본값=None)

In [64]:
train_rmse_total = []
test_rmse_total = []

for train_index, test_index in kf.split(X):
  # X_train, X_test, y_train, y_test 정의
  X_train, X_test = X.loc[train_index], X.loc[test_index]  
  y_train, y_test = y[train_index], y[test_index]  

  model = RandomForestRegressor(n_estimators=300, max_depth=50, min_samples_split=5, min_samples_leaf=1, n_jobs=-1, random_state=100)  # 하이퍼파라미터 지정
  model.fit(X_train, y_train)
  train_pred = model.predict(X_train)
  test_pred = model.predict(X_test)

  train_rmse = mean_squared_error(y_train, train_pred) ** 0.5  # train set rmse
  test_rmse = mean_squared_error(y_test, test_pred) ** 0.5  # test set rmse

  train_rmse_total.append(train_rmse)
  test_rmse_total.append(test_rmse)

In [65]:
print('train_rmse:', sum(train_rmse_total)/5, 'test_rmse:', sum(test_rmse_total)/5)

train_rmse: 66762.84568886801 test_rmse: 142205.83441414658


train rmse는 하이퍼파라미터 튜닝 전보다 다소 높아졌으나, test rmse는 조금 더 낮아짐
- 조금이나마 오버피팅이 줄어들었으니 새로운 데이터를 예측하기에는 더 좋은 모델이라고 할 수 있음